In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
train_transform = transforms.Compose([
    # Random augmentations (different each time!) 
    transforms .RandomHorizontalFlip(p=0.5), 
    transforms .RandomRotation(degrees=10), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

In [4]:
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=train_transform
)

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=val_transform
)

100%|██████████| 170M/170M [00:02<00:00, 77.1MB/s]


In [5]:
train_loader =  DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4, #Permite que dois núcleos da CPU preparem os dados simultaneamente.
    pin_memory=True
               
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4, #Permite que dois núcleos da CPU preparem os dados simultaneamente.
    pin_memory=True
)

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 🔹 Primeiro bloco convolucional
        self.conv1 = nn.Conv2d(
            in_channels=3,    # RGB
            out_channels=32,
            kernel_size=3,
            padding=1
        )
        self.bn1   = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # 🔹 Segundo bloco convolucional
        self.conv2 = nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=3,
            padding=1
        )
        self.bn2   = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # 🔹 Flatten
        self.flatten = nn.Flatten()

        # 🔹 Camadas totalmente conectadas
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.dropout = nn.Dropout(0.5)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Entrada: (batch, 3, 32, 32)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        # → (batch, 32, 16, 16)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        # → (batch, 64, 8, 8)

        x = self.flatten(x)
        # → (batch, 4096)

        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x) 
        x = self.fc2(x)

        return x

In [7]:
model = SimpleCNN().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [8]:
for epoch in range(50):
    print("training epoch: ", epoch)
    running_loss = 0.0
    model.train()
    

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print("loss:", running_loss/len(train_loader))

training epoch:  0
loss: 1.6921684240441188
training epoch:  1
loss: 1.4628998727139915
training epoch:  2
loss: 1.384110320254665
training epoch:  3
loss: 1.3382207009645983
training epoch:  4
loss: 1.3015545798689507
training epoch:  5
loss: 1.2660251333737922
training epoch:  6
loss: 1.234050834803935
training epoch:  7
loss: 1.2009383106170712
training epoch:  8
loss: 1.1795240137583154
training epoch:  9
loss: 1.1494226167573953
training epoch:  10
loss: 1.1149025605157818
training epoch:  11
loss: 1.087260862598029
training epoch:  12
loss: 1.0638878811198427
training epoch:  13
loss: 1.0356765718716185
training epoch:  14
loss: 1.0097428249276204
training epoch:  15
loss: 0.9894416597493164
training epoch:  16
loss: 0.9707262596053541
training epoch:  17
loss: 0.9557772380921542
training epoch:  18
loss: 0.9354154388313098
training epoch:  19
loss: 0.9223725884924154
training epoch:  20
loss: 0.904765453484967
training epoch:  21
loss: 0.891882352893005
training epoch:  22
loss:

In [9]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        images = image.to(device)
        labels = label.to(device)
        output = model(images)
        _, predicted = output.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    image, label = test_loader.dataset[0]
    images = image.to(device)
    images = images.unsqueeze(0) 
    with torch.no_grad():
        output = model(images)
        _, predicted = torch.max(output, 1)
        
print("Classe real:     ", classes[label])
print("Classe predita:  ", classes[predicted.item()])

print("Accuracy:", 100 * correct / total)

    

Classe real:      cat
Classe predita:   cat
Accuracy: 76.87
